# 🦺 PPE Detection using YOLOv8
Roboflow에서 제공하는 PPE 데이터셋을 기반으로 YOLOv8 모델을 학습하고, 개인 보호구 착용 여부를 감지합니다.

In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import files
uploaded = files.upload()  # 여기서 PPE.v2i.yolov8.zip 파일 업로드

In [ ]:
import zipfile
with zipfile.ZipFile("PPE.v2i.yolov8.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/PPE.v2i.yolov8")

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model.train(
    data="/content/PPE.v2i.yolov8/data.yaml",
    epochs=30,
    imgsz=640,
    batch=16
)

In [ ]:
from google.colab import files
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import os

# 1. 이미지 업로드
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print(f"\n📸 선택된 이미지: {image_path}")

# 2. 모델 로드
model = YOLO("/content/runs/detect/train/weights/best.pt")

# 3. 예측 실행 (📂 디렉토리 지정해서 저장 확정적으로 하게끔)
results = model.predict(
    source=f"/content/{image_path}",
    conf=0.3,
    save=True,
    save_txt=False,
    name="predict_final",  # 이 폴더로 고정
    exist_ok=True  # 덮어쓰기 허용
)

# 4. 결과 분석 (미착용 여부 판단)
for result in results:
    names = result.names
    boxes = result.boxes
    labels = [names[int(cls)] for cls in boxes.cls]

    print(f"\n🎯 감지된 객체: {labels}")

    if "Person" in labels:
        missing_items = []
        if "Helmet" not in labels:
            missing_items.append("헬멧")
        if "Vest" not in labels:
            missing_items.append("조끼")
        if "Boots" not in labels:
            missing_items.append("안전화")

        if missing_items:
            print(f"🚨 미착용 감지됨! 미착용 항목: {', '.join(missing_items)}")
        else:
            print("✅ 모든 보호구 착용 완료!")
    else:
        print("⚠️ 사람이 감지되지 않았습니다.")

# 5. 예측 이미지 시각화
base_name = os.path.splitext(image_path)[0]
predicted_path = f"/content/runs/detect/predict_final/{base_name}.jpg"

# 🔍 저장된 이미지 목록도 출력 (문제 디버깅용)
print("\n📁 저장된 파일 목록:")
print(os.listdir("/content/runs/detect/predict_final"))

# 시각화
if os.path.exists(predicted_path):
    img = Image.open(predicted_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title("📷 PPE 예측 결과")
    plt.show()
else:
    print("❗ 예측 이미지가 저장되지 않았습니다.")


In [ ]:
from google.colab import files
files.download('/content/runs/detect/train/weights/best.pt')
files.download('/content/runs/detect/train/results.png')